In [1]:
!nvidia-smi

Mon Sep 11 19:37:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/beratcmn/prompter-llama-2-7b
%cd ./prompter-llama-2-7b

Cloning into 'prompter-llama-2-7b'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 3), reused 8 (delta 2), pack-reused 0
Receiving objects: 100% (13/13), 32.29 KiB | 10.76 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/prompter-llama-2-7b


In [3]:
%pip install bitsandbytes transformers peft accelerate datasets loralib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00


In [4]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
notebook_login()

In [6]:
MODEL_NAME = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [7]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [8]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [10]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [11]:
modules = find_all_linear_names(model)
modules

['down_proj', 'k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj']

In [12]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 39976960 || all params: 3540389888 || trainables%: 1.1291682911958425


### Testing the original model

In [13]:
prompt = """
<human>: Act as a Linux Terminal
<assistant>:
""".strip()

In [14]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [15]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: Act as a Linux Terminal
<assistant>: Act as a Linux Assistant
<server>: Act as a Linux Server
<web>: Act as a Linux Web Server
<web-proxy>: Act as a Linux Web Proxy
<web-proxy-ssl>: Act as a Linux Web Proxy SSL
<web-proxy-ssl-https>: Act as a Linux Web Proxy SSL HTTPS
<web-proxy-ssl-https-tls12>: Act as a Linux Web Proxy SSL HTTPS TLS1.2
<web-proxy-ssl-https-tls13>: Act as a Linux Web Proxy SSL HTTPS TLS1.3
<web-proxy-ssl-https-tls13-tls12>: Act as a Linux Web Proxy SSL HTTPS TLS1.3 TLS1.2
<web-proxy-ssl-https-tls13-tls12-tls11>: Act as a Linux Web
CPU times: user 24.5 s, sys: 335 ms, total: 24.8 s
Wall time: 28.7 s


### Setting the data

In [16]:
data = load_dataset("csv", data_files="./data/prompts.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
data

DatasetDict({
    train: Dataset({
        features: ['User', 'Prompt'],
        num_rows: 164
    })
})

In [18]:
data["train"][0]

{'User': 'Act as a Linux Terminal',
 'Prompt': 'I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. When I need to tell you something in English, I will do so by putting text inside curly brackets {like this}. My first command is pwd'}

In [19]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["User"]}
<assistant>: {data_point["Prompt"]}
""".strip()

In [20]:
def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [21]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### Finetuning

In [22]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=5,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.640900
2,2.652300
3,2.925200
4,2.953300
5,2.696400
6,2.659200
7,2.518200
8,2.661300
9,2.042500
10,2.197500


TrainOutput(global_step=205, training_loss=0.874540976671184, metrics={'train_runtime': 1273.7026, 'train_samples_per_second': 0.644, 'train_steps_per_second': 0.161, 'total_flos': 1971438068736000.0, 'train_loss': 0.874540976671184, 'epoch': 5.0})

### Saving the model

In [23]:
model.save_pretrained("trained-model")

In [24]:
PEFT_MODEL = "beratcmn/prompter-llama-2-7b-0.1-5e"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:844: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/160M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/beratcmn/prompter-llama-2-7b-0.1-5e/commit/7b223e5715070fedfe73a6c2fd883c9c69385c51', commit_message='Upload model', commit_description='', oid='7b223e5715070fedfe73a6c2fd883c9c69385c51', pr_url=None, pr_revision=None, pr_num=None)

### Testing the finetuned model

In [25]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [26]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [27]:
%%time
device = "cuda:0"

prompt = """
<human>: Act as cheap travel ticket advisor
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: Act as cheap travel ticket advisor
<assistant>: I want you to act as a cheap travel ticket advisor. I will tell you my departure city and you will suggest a destination, not necessarily a place, that has the cheapest flights from my departure city. You will also provide a list of the cheapest flights to that destination. My first request is “I am leaving from Istanbul and I want to visit London.” Please wait for my next request to continue. Do not write explanations, just reply with the list of flights and the destination you recommend. My first request is “Istanbul to London” and the list of flights is [...]. My next request is “I am leaving from Istanbul and I want to visit Paris.” Please wait for my next request to continue. Do not write explanations, just reply with the list of flights and the destination you recommend. My first request is “Istanbul to Paris” and the list of flights is [...]. My next
CPU times: user 24.7 s, sys: 274 ms, total: 24.9 s
Wall time: 25.2 s


In [28]:
%%time
device = "cuda:0"

prompt = """
<human>: Act as a student career guide
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Act as a student career guide
<assistant>: I want you to act as a student career guide. I will provide you with a student interested in pursuing a particular profession and your task is to suggest what that student should study in order to achieve their goals. You should only reply with the recommended study program, and nothing else. Do not write explanations. My first request is “I want to become a software developer, what should I study?”
Act as a student career guide
I want you to act as a student career guide. I will provide you with a student interested in pursuing a particular profession and your task is to suggest what that student should study in order to achieve their goals. You should only reply with the recommended study program, and nothing else. Do not write explanations. My first request is “I want to become a software developer, what should I study?”
Reply “Software engineering”
I want you to act as a student career guide. I will provide you with a student inte

In [29]:
%%time
device = "cuda:0"

prompt = """
<human>: Act as a newborn parent tutor
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Act as a newborn parent tutor
<assistant>: I want you to act as a newborn parent tutor. I will provide you with a baby and some basic instructions on how to take care of them. Your task is to teach the parents how to properly feed, diaper, and comfort the baby. You should also instruct them on safety precautions they should take while handling the infant, and other related topics. My first request is “I need help teaching my husband and I how to properly care for our newborn son.” Please keep in mind that the parents are not experienced in taking care of infants, so your instructions should be easy to follow and not too technical. My first question is “How often should we be changing our son’s diaper?” Your response should include a suggestion for an appropriate frequency, as well as tips on how to properly clean and dispose of the waste. My second question is “What should we do if our son cries a lot and doesn’t seem to be comforted by our attempts
CPU times: user 25.3 s, sys

In [35]:
%%time
device = "cuda:0"

prompt = """
<human>: Act as a helpful friend for a 9 year old boy
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Act as a helpful friend for a 9 year old boy
<assistant>: I want you to act as a helpful friend for a 9 year old boy. I will ask you questions and you will provide the boy with helpful advice or suggestions on how to approach the problem. Do not write any explanations or additional words, just reply with the advice or suggestion. My first request is “I need help on solving a mathematical problem, can you please provide a solution?”. Reply “Try using this formula, (insert formula here). This should solve your problem.”. My first sentence is “Hi there, I need help on solving a mathematical problem, can you please provide a solution?”.
Act as a helpful friend for a 9 year old boy
I need help on solving a mathematical problem, can you please provide a solution?
Try using this formula, (insert formula here). This should solve your problem.
Hi there, I need help on solving a mathematical problem, can you please provide a solution?
Act as a helpful friend for a
CPU times: user 31.9 s

In [33]:
%%time
device = "cuda:0"

prompt = """
<human>: Act as an effective altruist mentor
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Act as an effective altruist mentor
<assistant>: I want you to act as an effective altruist mentor. I will provide you with a student interested in learning about effective altruism and your task is to help the student develop an understanding of effective principles, strategies for maximizing their impact, and suggestions for further study or action. You should use your best judgment and experience to create a plan that fits the student’s interests and abilities. My first request is “I want to become a more effective philanthropist.” Your task is to create a plan for the student that focuses on understanding how their money can do the most good, researching charities carefully, and investing in long-term solutions rather than short-term projects. My second request is “I want to do something meaningful with my life.” Your task is to help the student explore careers that have a positive impact on society, develop skills relevant to those fields, and create goals focused on maki

In [34]:
%%time
device = "cuda:0"

prompt = """
<human>: Act as a chef who mainly cooks fast food
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Act as a chef who mainly cooks fast food
<assistant>: I want you to act as a chef who mainly cooks fast food. You will be responsible for developing recipes for popular dishes such as hamburgers, fried chicken, pizza and tacos. You will also need to consider cost-effectiveness and health concerns when creating your dishes. My first suggestion request is “I need help developing a healthy alternative to fried foods that can be served in a fast food restaurant”. Please respond with the recipe for the dish, as well as an estimate of how many servings it will make. My first suggestion request is “I need help developing a healthy alternative to fried foods that can be served in a fast food restaurant”. Please respond with the recipe for the dish, as well as an estimate of how many servings it will make. My first suggestion request is “I need help developing a healthy alternative to fried foods that can be served in a fast food
CPU times: user 27.9 s, sys: 174 ms, total: 28.1 s
Wall 

In [37]:
%%time
device = "cuda:0"

prompt = """
<human>: Act as a doctor who is also a fireman
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Act as a doctor who is also a fireman
<assistant>: I want you to act as a doctor who is also a fireman. I will provide you with details of a patient needing medical attention and you will use your knowledge of both medicine and fire safety to determine the best course of action. You should be able to recommend medicines, herbs or other natural alternatives for treating illnesses; perform basic first aid procedures such as bandaging wounds or administering medicines; advise on safety precautions for preventing fires in the home; and suggest emergency procedures in case of fire outbreaks. My first request is “I need help treating a case of severe abdominal pain.” What should I do?
Act as a Doctor Who is Also a Fireman
I want you to act as a doctor who is also a fireman. I will provide you with details of a patient needing medical attention and you will use your knowledge of both medicine and fire safety to determine the best course of action. You
CPU times: user 26.3 s, sys: 204

In [38]:
%%time
device = "cuda:0"

prompt = """
<human>: Act as a C# interactive shell
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Act as a C# interactive shell
<assistant>: I want you to act as a C# interactive shell. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. Do not write explanations. Do not type commands unless I instruct you to do so. When I need to tell you something in english, I will do so by putting text inside curly brackets {like this}. My first command is pwd. Reply with the output of pwd inside one unique code block.
<assistant>: I want you to only reply with the terminal output inside one unique code block, and nothing else. Do not write explanations. Do not type commands unless I instruct you to do so. When I need to tell you something in english, I will do so by putting text inside curly brackets {like this}. My first command is pwd. Reply with the output of pwd inside one unique code
CPU times: user 26.9 s, sys: 169 ms, total: 27.1 s
Wall time: 30.8 s


In [39]:
%%time
device = "cuda:0"

prompt = """
<human>: Act as a drunk philosopher
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Act as a drunk philosopher
<assistant>: I want you to act as a drunk philosopher. You will provide me with entertaining and thought-provoking ideas, but your responses may be nonsensical or illogical due to your intoxication. Additionally, you should only reply with your thoughts and opinions; do not write explanations. My first request is “I need an idea that makes me question everything.” Your response should be “Everything is meaningless.” My second request is “I need a different idea that makes me question the universe.” Your response should be “The universe is a giant cuckoo clock and we are all just little birds.” My third request is “I need an idea that makes me laugh.” Your response should be “Why is the number 42 funny?” My turn to ask a question - “What is the meaning of life?”
Act as a drunk philosopher
I want you to act as a drunk philosopher. You will provide
CPU times: user 26.7 s, sys: 110 ms, total: 26.8 s
Wall time: 29.5 s
